In [1]:
import os
os.chdir("D://Course code")

import pandas as pd
import numpy as np



In [4]:
df = pd.read_csv('diabetes.csv')
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [8]:
print(df[df['Glucose']==0])


     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
75             1        0             48             20        0  24.7   
182            1        0             74             20       23  27.7   
342            1        0             68             35        0  32.0   
349            5        0             80             32        0  41.0   
502            6        0             68             41        0  39.0   

     DiabetesPedigreeFunction  Age  Outcome  
75                      0.140   22        0  
182                     0.299   21        0  
342                     0.389   22        0  
349                     0.346   37        1  
502                     0.727   41        1  


In [10]:
#### Filling the missing values i.e 0 with median
df['Insulin'] = np.where(df['Insulin']==0, df['Insulin'].median(), df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0, df['SkinThickness'].median(), df['SkinThickness'])
df['Glucose']=np.where(df['Glucose']==0, df['Glucose'].median(), df['Glucose'])

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [11]:
#### Seperating target and attribute
y = df.pop('Outcome')
X = df

In [17]:
#### Creating train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=123) 

(154, 8)

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf_rfc = RandomForestClassifier(n_estimators = 50)

clf_rfc.fit(X_train, y_train)
predictions = clf_rfc.predict(X_test)



In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.8051948051948052
[[86 10]
 [20 38]]
              precision    recall  f1-score   support

           0       0.81      0.90      0.85        96
           1       0.79      0.66      0.72        58

    accuracy                           0.81       154
   macro avg       0.80      0.78      0.78       154
weighted avg       0.80      0.81      0.80       154



In [27]:
n_estimators = [int(x) for x in np.linspace(start=20 , stop=100 , num = 5)] 
max_depth = [int(x) for x in np.linspace(20,100,5)]
max_leaf_nodes = [int(x) for x in np.linspace(20,100,10)]

In [37]:
#### Hyperparameter tuning using randomized search selecting different parameters
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier()
parameters = {'n_estimators' : n_estimators,
             'criterion': ['gini', 'entropy'],
              'max_depth' : max_depth,
              'min_samples_split': [2,3,4,5,6,7],
              'min_samples_leaf' : [1,2,3,4,5,6],
              'max_features' : ['auto' , 'sqrt' , 'log2'],
              'max_leaf_nodes' : max_leaf_nodes
             }
              


In [44]:
#### Doing randomized search and fitting on the training data
random_search = RandomizedSearchCV(estimator = rf, param_distributions = parameters, n_iter = 100 , cv=3 , verbose =2 ,
                                    n_jobs=-1, random_state = 130)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   25.9s finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [20, 40, 60, 80, 100],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'max_leaf_nodes': [20, 28, 37, 46, 55,
                                                           64, 73, 82, 91,
                                                           100],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7],
                                        'n_estimators': [20, 40, 60, 80, 100]},
                   random_state=130, verbose=2)

In [48]:
#### Gives the best parameters
random_search.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=80, max_leaf_nodes=20,
                       min_samples_leaf=5, min_samples_split=7,
                       n_estimators=60)

In [49]:
#### Creating model with beat parms according to randomized search 
model_best = random_search.best_estimator_


In [50]:
#### Making predictions
pred_tuned = model_best.predict(X_test)

In [51]:
print(confusion_matrix(y_test, pred_tuned))
print(accuracy_score(y_test, pred_tuned))
print(classification_report(y_test, pred_tuned))

[[85 11]
 [23 35]]
0.7792207792207793
              precision    recall  f1-score   support

           0       0.79      0.89      0.83        96
           1       0.76      0.60      0.67        58

    accuracy                           0.78       154
   macro avg       0.77      0.74      0.75       154
weighted avg       0.78      0.78      0.77       154



In [46]:
#### Gives best parameters 
best_parameters = random_search.best_params_
best_parameters

{'n_estimators': 60,
 'min_samples_split': 7,
 'min_samples_leaf': 5,
 'max_leaf_nodes': 20,
 'max_features': 'auto',
 'max_depth': 80,
 'criterion': 'entropy'}

In [63]:
#### Performing gridsearch with the narrowed down parameters(best_parameters)
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()

gs_parms = {'n_estimators': [best_parameters['n_estimators']-10, 
                            best_parameters['n_estimators']-20,
                            best_parameters['n_estimators']-30,
                            best_parameters['n_estimators']-40,
                            best_parameters['n_estimators']-50],
            'min_samples_split': [7,8,9,10],
            'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10],
            'max_leaf_nodes':[best_parameters['max_leaf_nodes'] + 10,
                              best_parameters['max_leaf_nodes']+20,
                              best_parameters['max_leaf_nodes']+30,
                              best_parameters['max_leaf_nodes']-10,
                              best_parameters['max_leaf_nodes']+50],
            'max_features': [best_parameters['max_features']],
            'max_depth' : [best_parameters['max_depth'] +20,
                          best_parameters['max_depth'] +40,
                          best_parameters['max_depth'] -20,
                          best_parameters['max_depth'] -40],
            'criterion' : [best_parameters['criterion']]
                            }

In [60]:
print(gs_parms)

{'n_estimators': [50, 40, 30, 20, 10], 'min_samples': [7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_leaf_nodes': [30, 40, 50, 10, 70], 'max_features': ['auto'], 'max_depth': [100, 120, 60, 40], 'criterion': ['entropy']}


In [64]:
#### Fit on the data
gs_model = GridSearchCV(estimator = rf, param_grid=gs_parms, n_jobs=-1, cv=10, verbose=2)
gs_model.fit(X_train,y_train)

Fitting 10 folds for each of 4000 candidates, totalling 40000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 658 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 1224 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 1954 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 2844 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 3898 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 5112 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 6490 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 8028 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 9730 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 11592 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 13618 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 15804 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 18154 tasks   

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'],
                         'max_depth': [100, 120, 60, 40],
                         'max_features': ['auto'],
                         'max_leaf_nodes': [30, 40, 50, 10, 70],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [7, 8, 9, 10],
                         'n_estimators': [50, 40, 30, 20, 10]},
             verbose=2)

In [66]:
#### Find best parameters for the model
gs_tuned_model = gs_model.best_estimator_

In [68]:
#### See the score, confusion_matrix, classification_report after aking predictions
gs_predictions = gs_tuned_model.predict(X_test)
print(accuracy_score(y_test, gs_predictions))
print(confusion_matrix(y_test, gs_predictions))
print(classification_report(y_test, gs_predictions))

0.7597402597402597
[[82 14]
 [23 35]]
              precision    recall  f1-score   support

           0       0.78      0.85      0.82        96
           1       0.71      0.60      0.65        58

    accuracy                           0.76       154
   macro avg       0.75      0.73      0.74       154
weighted avg       0.76      0.76      0.76       154

